In [4]:
!git clone https://github.com/MrudulaSatya/imagesTestRecn2d.git
!git clone https://github.com/MrudulaSatya/imagesTestn2d.git
!git clone https://github.com/MrudulaSatya/night2day.git
!git clone https://github.com/MrudulaSatya/testn2d.git

fatal: destination path 'imagesTestRecn2d' already exists and is not an empty directory.
fatal: destination path 'imagesTestn2d' already exists and is not an empty directory.
fatal: destination path 'night2day' already exists and is not an empty directory.
fatal: destination path 'testn2d' already exists and is not an empty directory.


In [0]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B


    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)


In [7]:
import os

os.makedirs('test/%s' % dataset_name, exist_ok=True)
r, c = 3, 3
epoch = 0
for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size=1,is_testing=True)):
  gen_imgs = np.concatenate([imgs_A])

  # Rescale images 0 - 
  gen_imgs = 0.5 * gen_imgs + 0.5

  titles = ['Generated', 'Generated', 'Original']
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt])
      axs[i, j].set_title(titles[i])
      axs[i,j].axis('off')
      cnt += 1
      break
    break
  fig.savefig("test/%d_%d_rec.png" % (epoch, batch_i))
  plt.close()
  epoch += 1

NameError: ignored

In [8]:
import os
print( os.getcwd() )
print( os.listdir('test') )
!zip -r /content/test.zip /content/test
from google.colab import files
files.download("/content/test.zip")   

/content


FileNotFoundError: ignored

In [12]:
import matplotlib.image as mpimg
import os
import numpy as np
import tensorflow as tf
# First, load the image
averageRec = 0
average2 = 0
averageSRec = 0
averageS2 = 0
averagePRec = 0
averageP2 = 0
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)
 # Configure data loader
dataset_name = 'night2day'
data_loader = DataLoader(dataset_name=dataset_name, img_res=(img_rows, img_cols))
for batch_i in range(0,99):
  filenameRec = "imagesTestRecn2d/%d_%d_rec.png" % (batch_i, batch_i)
  filenameOrig = "imagesTestn2d/%d_%d_rec.png" % (batch_i, batch_i)
  filenameA = "testn2d/%d_%d_rec.png" % (batch_i, batch_i)
  # Load the image
  imageRec = mpimg.imread(filenameRec)
  imageOrig = mpimg.imread(filenameOrig)
  imgs_A = mpimg.imread(filenameA)
  # Print out its shape
  print(imageRec.shape)
  print(imageOrig.shape)
  print(imgs_A.shape)
  
  
  imageRecT = tf.image.convert_image_dtype(imageRec, tf.float32)
  imageOrigT = tf.image.convert_image_dtype(imageOrig, tf.float32)
  imgs_AT = tf.image.convert_image_dtype(imgs_A, tf.float32)
  mseRec = (np.square(np.abs(np.subtract(imgs_A, imageRec)))).mean(axis=None) #rec
  mse2 = (np.square(np.abs(np.subtract(imgs_A, imageOrig)))).mean(axis=None) #original
  ssimRec = tf.image.ssim(imgs_AT, imageRecT, max_val=1.0) #rec
  ssim2 = tf.image.ssim(imgs_AT, imageOrigT, max_val=1.0) #original
  psnrRec = tf.image.psnr(imgs_AT, imageRecT, max_val=1.0) #rec
  psnr2 = tf.image.psnr(imgs_AT, imageOrigT, max_val=1.0) #original
  averageRec = averageRec + mseRec
  average2 = average2 + mse2
  averageSRec = averageSRec + ssimRec
  averageS2 = averageS2 + ssim2
  averagePRec = averagePRec + psnrRec
  averageP2 = averageP2 + psnr2
  
averageRec = averageRec/99
average2 = average2/99
averageSRec = averageSRec/99
averageS2 = averageS2/99
averagePRec = averagePRec/99
averageP2 =  averageP2/99
print(averageRec) #rec
print(average2) #original
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(averageSRec.eval()) #rec
    print(averageS2.eval()) #orig
    print(averagePRec.eval()) #rec
    print(averageP2.eval()) #orig
  
  

(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 432, 4)
(288, 